##### Utbildningsanordnare tabellen:
- Beviljade platser totalt
- Beviljandegrad %
- Beviljade kurser

Tabellen är sorterad efter:
- beviljade antal platser totalt, vilket innebär att vissa anordnare fått högre värde på andra kolumner, men lägre plats i tabellen. 
- TO DO: Det maximala beviljade statsbidraget är baserade på antalet beviljade platser, hur mycket som är beviljat för en plats och fördelade över utbildningsomgångarna. Notera att beloppet varierar beroende på vilken beviljad utbildning.

In [4]:
def load_course_data(folder_path, year):
    """
    Loads and merges course data from both 'resultat' and 'inkomna' Excel files for the given year.
    Returns a merged pandas DataFrame or None if files not found.
    
    Parameters:
        folder_path: Path to folder containing the Excel files
        year: Year to filter files by
    
    Returns:
        Merged DataFrame containing data from both files
    """
    year_str = str(year)
    files = os.listdir(folder_path)
    
    # Find result file
    result_files = [
        f for f in files
        if year_str in f and 'resultat' in f.lower() and f.endswith('.xlsx')
    ]
    
    # Find incoming applications file
    incoming_files = [
        f for f in files
        if year_str in f and 'inkomna' in f.lower() and f.endswith('.xlsx')
    ]
    
    # Validate file existence
    if not result_files or not incoming_files:
        print(f"Missing required files for year {year}")
        return None
    
    if len(result_files) > 1 or len(incoming_files) > 1:
        print(f"Warning: Multiple matching files found for year {year}")
        return None
    
    # Load result data
    result_path = os.path.join(folder_path, result_files[0])
    print(f"Loading results from: {result_files[0]}")
    df_result = pd.read_excel(result_path)
    
    # Load incoming applications data
    incoming_path = os.path.join(folder_path, incoming_files[0])
    print(f"Loading applications from: {incoming_files[0]}")
    df_incoming = pd.read_excel(incoming_path)
    
    # Get columns starting with 'Sökt antal platser'
    place_columns = [col for col in df_incoming.columns if col.startswith('Sökt antal platser')]
    
    # Merge dataframes on Diarienummer
    merged_df = pd.merge(
        df_result,
        df_incoming[['Diarienummer'] + place_columns],
        on='Diarienummer',
        how='left',
        validate='1:1'  # Ensure one-to-one merge
    )
    
    # Validate merge results
    unmatched = merged_df['Diarienummer'].isna().sum()
    if unmatched > 0:
        print(f"Warning: {unmatched} records could not be matched between files")
    
    print(f"Successfully loaded data with {len(merged_df)} rows and {len(merged_df.columns)} columns")
    return merged_df

# Example usage:
folder = '../data/resultat_kurser'
selected_year = 2025
df = load_course_data(folder, selected_year)

if df is not None:
    print("\nColumns from incoming applications file:")
    place_cols = [col for col in df.columns if col.startswith('Sökt antal platser')]
    for col in place_cols:
        print(f"- {col}")

Loading results from: resultat-2025-for-kurser-inom-yh.xlsx
Loading applications from: inkomna-ansokningar-2025-for-kurser.xlsx
Successfully loaded data with 1026 rows and 17 columns

Columns from incoming applications file:
- Sökt antal platser 2025
- Sökt antal platser 2026


In [5]:
df.columns

Index(['Diarienummer', 'Beslut', 'Anordnare namn', 'Utbildningsnamn',
       'Utbildningsområde', 'Antal beviljade platser start 2025',
       'Antal beviljade platser start 2026', 'Totalt antal beviljade platser',
       'YH-poäng', 'Kommun', 'Län', 'FA-region', 'Antal kommuner', 'Antal län',
       'Antal FA-regioner', 'Sökt antal platser 2025',
       'Sökt antal platser 2026'],
      dtype='object')

##### Add a column where funding will be calculated based on the YH credits AND educational area:
- 

In [7]:
import duckdb

query = """--sql
SELECT 
    "Anordnare namn",

    -- Sum of total granted places
    SUM("Totalt antal beviljade platser") AS "Beviljade platser totalt",

    -- Approval rate (%): total granted places / total applied places
    ROUND(
        100.0 * SUM("Totalt antal beviljade platser") / 
        NULLIF(SUM("Sökt antal platser 2025" + "Sökt antal platser 2026"), 0)
    , 1) AS "Beviljandegrad %",

    -- Count of approved programs
    COUNT(CASE WHEN "Beslut" = 'Beviljad' THEN 1 END) AS "Beviljade utbildningar"

FROM df

GROUP BY "Anordnare namn"
ORDER BY "Beviljade platser totalt" DESC
"""

# Execute query and convert to DataFrame
df_providers = duckdb.query(query).to_df()

# Display results with formatting
print("Utbildningsanordnare statistik:")
display(df_providers.style
       .format({
           'Beviljade platser totalt': '{:,.0f}',
           'Beviljandegrad %': '{:.1f}%',
           'Beviljade kurser': '{:,.0f}'
       })
       .hide(axis='index'))

Utbildningsanordnare statistik:


Anordnare namn,Beviljade platser totalt,Beviljandegrad %,Beviljade utbildningar
IHM Business School AB Göteborg,"2,990",84.9%,35
Nackademin AB,"1,945",82.6%,20
Medieinstitutet i Sverige AB,"1,659",83.9%,20
YH Akademin AB,"1,615",66.2%,35
Företagsuniversitetet AB,"1,215",79.7%,9
KompetensUtvecklingsInstitutet Sverige AB,"1,125",97.0%,16
Lexicon Yrkeshögskola AB,"1,039",53.2%,16
TUC Sweden AB - Yrkeshögskola,975,48.6%,16
XeDrix AB,960,80.7%,7
Stockholm School of Business,910,69.7%,11


##### Import a fianansing information per educational area for courses from:
- https://www.myh.se/yrkeshogskolan/ansok-om-att-bedriva-utbildning/ansokan-kurser/statsbidrag-och-schablonnivaer 

Statsbidraget utgår från schabloner där bidraget bestäms per studerandeplats i heltidsutbildning som omfattar 40 veckor och 200 yrkeshögskolepoäng (årsplats).

# TO DO: make sure that the format of data is correct, that all education areas are saved with " " AND that Utan momskompensation,Med momskompensation format is changed to int

In [11]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.myh.se/yrkeshogskolan/ansok-om-att-bedriva-utbildning/ansokan-kurser/statsbidrag-och-schablonnivaer"

def extract_tables(url):
    r = requests.get(url)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    # Locate section title
    header = soup.find(lambda tag: tag.name in ["h2", "h3"] and "Schablonnivåer i kronor" in tag.get_text())
    if not header:
        raise ValueError("Couldn't find 'Schablonnivåer i kronor' section")

    # Look for <pre> or <table>
    tables = []
    pre = header.find_next("pre")
    if pre:
        tables = parse_pre_text(pre.get_text())
    else:
        for tbl in header.find_all_next("table", limit=2):
            tables.append(parse_html_table(tbl))
    return tables

def parse_pre_text(text):
    lines = [ln.strip() for ln in text.strip().splitlines() if ln.strip()]
    sections = []
    curr = []
    for ln in lines:
        if ln.startswith("Utan") or ln.startswith("Med"):
            if curr:
                sections.append(curr)
                curr = []
        curr.append(ln)
    if curr:
        sections.append(curr)

    def to_df(section):
        title = section[0]
        rows = [ln.split() for ln in section[1:]]
        df = pd.DataFrame(rows[1:], columns=rows[0])
        df.attrs["title"] = title
        return df

    return [to_df(sec) for sec in sections]

def parse_html_table(table):
    rows = table.find_all("tr")
    data = []
    for row in rows:
        cols = [td.get_text(strip=True) for td in row.find_all(["td", "th"])]
        data.append(cols)
    df = pd.DataFrame(data[1:], columns=data[0])
    df.attrs["title"] = "Extracted HTML Table"
    return df

def save_combined_csv(dataframes, folder: str, filename: str):
    # Create folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)
    full_path = os.path.join(folder, filename)

    # Combine all dataframes with a source label
    combined = []
    for df in dataframes:
        title = df.attrs.get("title", "")
        df_with_title = df.copy()
        df_with_title.insert(0, "Source", title)
        combined.append(df_with_title)

    final_df = pd.concat(combined, ignore_index=True)
    final_df.to_csv(full_path, index=False)
    print(f"Saved combined table to: {full_path}")

if __name__ == "__main__":
    tables = extract_tables(URL)
    save_combined_csv(tables, folder="../data/resultat_kurser", filename="schablonnivaer.csv")


Saved combined table to: ../data/resultat_kurser\schablonnivaer.csv


'66 600'